In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading the dataframe
# https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv
vaccinations = pd.read_csv("./owid_downloaded/vaccinations.csv")

# Dropping columns not used
vaccinations.drop(columns=[
    'location', 
    'total_vaccinations_per_hundred', 
    'daily_vaccinations_raw', 
    'people_vaccinated_per_hundred', 
    'people_fully_vaccinated_per_hundred', 
    'daily_vaccinations_per_million'], inplace=True)

# Getting all iso_codes
iso_codes = vaccinations['iso_code'].unique()
iso_codes = [iso_code for iso_code in iso_codes if not 'OWID' in iso_code]


def get_people_fully_vaccinated_value(t, i, values):
    """Computes the `people_fully_vaccinated` where possible"""
    start = -1
    for val in values:
        if i >= val:
            start = val
        elif start >= 0 and i <= val:
            value = t['people_fully_vaccinated'][start] + ((i - start) * (t['people_fully_vaccinated'][val] - t['people_fully_vaccinated'][start])/(val-start))
            return value
    return None

df = pd.DataFrame(columns=vaccinations.columns)

for iso_code in iso_codes:
    country_data = vaccinations[vaccinations['iso_code'] == iso_code].copy()
    
    # Computing `total_vaccinations` and `people_fully_vaccinated` where possible
    total_vaccinations = country_data['total_vaccinations']
    people_fully_vaccinated = country_data['people_fully_vaccinated']
    people_vaccinated = country_data['people_vaccinated']
    daily_vaccinations = country_data['daily_vaccinations']
    
    for i in range(0, country_data.shape[0]):
        if i == 0:
            if not np.isnan(total_vaccinations.iloc[i]):
                daily_vaccinations.iloc[i] = total_vaccinations.iloc[i]
                people_vaccinated.iloc[i] = total_vaccinations.iloc[i]
            elif not np.isnan(daily_vaccinations.iloc[i]):
                total_vaccinations.iloc[i] = daily_vaccinations.iloc[i]
                people_vaccinated.iloc[i] = daily_vaccinations.iloc[i]
            elif not np.isnan(people_vaccinated.iloc[i]):
                total_vaccinations.iloc[i] = people_vaccinated.iloc[i]
                daily_vaccinations.iloc[i] = people_vaccinated.iloc[i]
            people_fully_vaccinated.iloc[i] = 0
        if i > 0 and np.isnan(total_vaccinations.iloc[i]):
            pred = total_vaccinations.iloc[i-1]
            total_vaccinations.iloc[i] = pred + country_data.iloc[i]['daily_vaccinations']
        
        if not np.isnan(country_data.iloc[i]['total_vaccinations']) and not np.isnan(country_data.iloc[i]['people_vaccinated']):
            people_fully_vaccinated.iloc[i] = country_data.iloc[i]['total_vaccinations'] - country_data.iloc[i]['people_vaccinated']
            
    country_data['total_vaccinations'] = total_vaccinations
    country_data['people_fully_vaccinated'] = people_fully_vaccinated
    country_data['daily_vaccinations'] = daily_vaccinations
    country_data['people_vaccinated'] = people_vaccinated
    
    values = list(country_data[~country_data['people_fully_vaccinated'].isna()].index)
    
    for i in range(1, country_data.shape[0]):
        if np.isnan(people_fully_vaccinated.iloc[i]):
            value = get_people_fully_vaccinated_value(country_data, people_fully_vaccinated.index[i], values)
            if value is not None:
                people_fully_vaccinated.iloc[i] = round(value)
        if np.isnan(people_vaccinated.iloc[i]):
            people_vaccinated.iloc[i] = country_data.iloc[i]['total_vaccinations'] - people_fully_vaccinated.iloc[i]
            
    country_data['people_fully_vaccinated'] = people_fully_vaccinated
    country_data['people_vaccinated'] = people_vaccinated
    
    df = df.append(country_data)

/Users/francescoposa/opt/anaconda3/envs/igh-whatif/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [3]:
df.to_csv('vaccinations.csv', index=False)